In [2]:
from nn import QLearningAgent, QNetwork
from verify_data import FishGame, ParseError

In [3]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            game = FishGame(f.readlines())
            memories + [game.memory(player) for player in game.players]
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-3_14:27.txt: Score is invalid [3, 5] [2, 6]
